Dependencias

In [18]:
!pip install pytrends #ignorar si ya esta instalado


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [19]:
!pip install pandas-ta #ignorar si ya esta instalado


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [20]:
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import pandas as pd
import os
import joblib
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import GridSearchCV 

In [21]:
possible_paths = [
    '../analisis_exploratorios_tp3/',  
    '../../analisis_exploratorios_tp3/'  
]

data_path = None
for path in possible_paths:
    if os.path.exists(f"{path}train.csv"):
        data_path = path
        break

if data_path:
    print(f" Archivos CSV encontrados en: {data_path}")
    train = pd.read_csv(f'{data_path}train.csv', index_col=0, parse_dates=True)
    validation = pd.read_csv(f'{data_path}validation.csv', index_col=0, parse_dates=True)
    test = pd.read_csv(f'{data_path}test.csv', index_col=0, parse_dates=True)
else:
    print(" No se encontraron los archivos CSV en ninguna ruta")
    print(" Contenido del directorio actual:")
    for item in os.listdir('.'):
        print(f"   {'' if os.path.isdir(item) else ''} {item}")

print(f" Datos cargados exitosamente:")
print(f"   Train: {train.shape}")
print(f"   Validation: {validation.shape}")
print(f"   Test: {test.shape}")
print(f"   Columnas disponibles: {len(train.columns)} columnas")


print(f"\n Primeras 10 columnas: {train.columns[:10].tolist()}")
print(f" Targets disponibles: {[col for col in train.columns if col.startswith('NextClose_BTC')]}")


print(f"\n Rango de fechas:")
print(f"   Train: {train.index.min()} a {train.index.max()}")
print(f"   Validation: {validation.index.min()} a {validation.index.max()}")
print(f"   Test: {test.index.min()} a {test.index.max()}")

 Archivos CSV encontrados en: ../analisis_exploratorios_tp3/
 Datos cargados exitosamente:
   Train: (1110, 48)
   Validation: (370, 48)
   Test: (370, 48)
   Columnas disponibles: 48 columnas

 Primeras 10 columnas: ['btc_c', 'btc_h', 'btc_l', 'btc_o', 'btc_v', 'sp500_c', 'dowj_c', 'nasdaq_c', 'eurostoxx_c', 'nikkei_c']
 Targets disponibles: ['NextClose_BTC1', 'NextClose_BTC2', 'NextClose_BTC3', 'NextClose_BTC4', 'NextClose_BTC5', 'NextClose_BTC6', 'NextClose_BTC7']

 Rango de fechas:
   Train: 2020-09-30 00:00:00 a 2025-10-23 00:00:00
   Validation: 2020-10-03 00:00:00 a 2025-10-19 00:00:00
   Test: 2020-10-14 00:00:00 a 2025-10-21 00:00:00


In [22]:
TARGETS_FUTUROS = [f"NextClose_BTC{i}" for i in range(1, 8)]
VARIABLES_A_EXCLUIR = TARGETS_FUTUROS + ['btc_c'] 

FULL_FEATURES_NO_FUTURE = [
    'btc_h', 'btc_l', 'btc_o', 'btc_v', 'sp500_c', 'dowj_c', 'nasdaq_c', 'eurostoxx_c', 'nikkei_c', 
    'day', 'month', 'Year', 'btc_prev_c1', 'btc_prev_c2', 'btc_prev_c3', 'btc_prev_c4',
    'btc_prev_c5', 'btc_prev_c6', 'btc_prev_c7', 'is_month_start', 'is_month_end',
    'btc_pct_prev1', 'btc_pct_prev2', 'btc_pct_prev3', 'btc_pct_prev4', 'btc_pct_prev5',
    'btc_pct_prev6', 'btc_pct_prev7', 'fng_value', 'bitcoin', 'RSI_14', 'BBL_20_2.0_2.0', 
    'BBM_20_2.0_2.0', 'BBU_20_2.0_2.0', 'BBB_20_2.0_2.0', 'BBP_20_2.0_2.0', 'MACD_12_26_9',
    'MACDh_12_26_9', 'MACDs_12_26_9', 'ATRr_14'
]

COMMON_FEATURES = [f for f in FULL_FEATURES_NO_FUTURE if f not in VARIABLES_A_EXCLUIR]
features_por_target = {t: COMMON_FEATURES for t in TARGETS_FUTUROS}
columnas_numericas = [
    'btc_h', 'btc_l', 'btc_o', 'btc_v', 'sp500_c', 'dowj_c', 'nasdaq_c', 'eurostoxx_c', 'nikkei_c',
    'btc_prev_c1', 'btc_prev_c2', 'btc_prev_c3', 'btc_prev_c4', 'btc_prev_c5', 'btc_prev_c6', 'btc_prev_c7',
    'fng_value', 'bitcoin', 'btc_pct_prev1', 'btc_pct_prev2', 'btc_pct_prev3', 'btc_pct_prev4', 'btc_pct_prev5',
    'btc_pct_prev6', 'btc_pct_prev7', 'RSI_14', 'BBL_20_2.0_2.0', 'BBM_20_2.0_2.0', 'BBU_20_2.0_2.0', 
    'BBB_20_2.0_2.0', 'BBP_20_2.0_2.0', 'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9', 'ATRr_14'
]
columnas_categoricas = ['day', 'month', 'Year', 'is_month_start', 'is_month_end']

param_grid_rf = {
    'regressor__n_estimators': [50, 100, 150],
    'regressor__max_depth': [10, 15, 20],
    'regressor__min_samples_leaf': [1, 3, 5]
}

rf_models_optimized = {} 
output_dir = 'modelos_entrenados'  
os.makedirs(output_dir, exist_ok=True)

print(" Iniciando BÚSQUEDA DE HIPERPARÁMETROS para 7 modelos Random Forest...")

for target in targets:
    print(f"\n--- Optimizando y Entrenando {target} ---")

    features_para_este_modelo = features_por_target[target]
    train_cleaned = train.dropna(subset=[target]) 

    X_train_subset = train_cleaned[features_para_este_modelo]
    y_train = train_cleaned[target]

    
    numeric_features = [col for col in features_para_este_modelo if col in columnas_numericas]
    categorical_features = [col for col in features_para_este_modelo if col in columnas_categoricas]
    
    numeric_transformer = Pipeline([('imputer', SimpleImputer(strategy='mean')), ('scaler', StandardScaler())])
    categorical_transformer = Pipeline([('imputer', SimpleImputer(strategy='most_frequent')), ('onehot', OneHotEncoder(handle_unknown='ignore'))])
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ],
        remainder='drop' 
    )
    
    full_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', RandomForestRegressor(random_state=42, n_jobs=-1))
    ])

    grid_search = GridSearchCV(
        estimator=full_pipeline, 
        param_grid=param_grid_rf, 
        scoring='neg_mean_absolute_error', 
        cv=3, 
        n_jobs=-1, 
        verbose=0
    )
    
    grid_search.fit(X_train_subset, y_train)
    best_rf_model = grid_search.best_estimator_
    rf_models_optimized[target] = best_rf_model
    
    X_val_subset = validation[features_para_este_modelo]
    y_true_full = validation[target] 

    y_pred_array = best_rf_model.predict(X_val_subset)
    y_pred_series = pd.Series(y_pred_array, index=X_val_subset.index)
    
    comparison_df = pd.DataFrame({'y_true': y_true_full, 'y_pred': y_pred_series})
    comparison_df.dropna(subset=['y_true'], inplace=True)
    
    y_true_aligned = comparison_df['y_true']
    y_pred_aligned = comparison_df['y_pred']
  
    rmse = np.sqrt(mean_squared_error(y_true_aligned, y_pred_aligned))
    mape = mean_absolute_percentage_error(y_true_aligned, y_pred_aligned) * 100
    
    print(f"    MEJORES PARÁMETROS: {grid_search.best_params_}")
    print(f"    Mejor Puntuación de CV (MAE): {-grid_search.best_score_:.2f} USD")
    print(f"    RMSE en Validación: {rmse:,.2f} USD, MAPE: {mape:.2f}%")

print(f"\n {len(rf_models_optimized)} modelos Random Forest optimizados y entrenados.")
joblib.dump(rf_models_optimized, os.path.join(output_dir, 'rf_modelos_optimizados.joblib'))
joblib.dump(features_por_target, os.path.join(output_dir, 'rf_features_por_target.joblib'))
print(f" Modelos y features guardados en: {output_dir}")

 Iniciando BÚSQUEDA DE HIPERPARÁMETROS para 7 modelos Random Forest...

--- Optimizando y Entrenando NextClose_BTC1 ---
    MEJORES PARÁMETROS: {'regressor__max_depth': 20, 'regressor__min_samples_leaf': 3, 'regressor__n_estimators': 100}
    Mejor Puntuación de CV (MAE): 1196.93 USD
    RMSE en Validación: 1,827.62 USD, MAPE: 2.43%

--- Optimizando y Entrenando NextClose_BTC2 ---
    MEJORES PARÁMETROS: {'regressor__max_depth': 20, 'regressor__min_samples_leaf': 1, 'regressor__n_estimators': 150}
    Mejor Puntuación de CV (MAE): 1562.16 USD
    RMSE en Validación: 2,175.62 USD, MAPE: 2.86%

--- Optimizando y Entrenando NextClose_BTC3 ---
    MEJORES PARÁMETROS: {'regressor__max_depth': 15, 'regressor__min_samples_leaf': 1, 'regressor__n_estimators': 150}
    Mejor Puntuación de CV (MAE): 1776.72 USD
    RMSE en Validación: 2,268.18 USD, MAPE: 3.13%

--- Optimizando y Entrenando NextClose_BTC4 ---
    MEJORES PARÁMETROS: {'regressor__max_depth': 15, 'regressor__min_samples_leaf': 1, '

In [23]:
try:
    test = pd.read_csv('test.csv')
except:
    print("Error: No se encuentra 'test.csv'.")

print("\nResultados Finales en TEST (Usando Mejores Hiperparámetros):")
print("-" * 60)

for target in targets:
    # A. Recuperar features
    features_para_este_modelo = features_por_target[target]
    
    # B. Preparar datos de test
    test_cleaned = test.dropna(subset=[target])
    X_test_subset = test_cleaned[features_para_este_modelo]
    y_test_true = test_cleaned[target]
    
    # C. ¡AQUÍ ESTÁ EL CAMBIO! Usamos rf_models_optimized en lugar de rf_models
    if target in rf_models_optimized:
        model = rf_models_optimized[target]  # <--- Usamos el modelo optimizado
        
        # D. Predecir
        y_test_pred = model.predict(X_test_subset)
        
        # E. Métricas
        rmse_test = np.sqrt(mean_squared_error(y_test_true, y_test_pred))
        mape_test = mean_absolute_percentage_error(y_test_true, y_test_pred) * 100
        
        print(f"Target: {target:15} | RMSE Test: {rmse_test:,.2f} | MAPE Test: {mape_test:.2f}%")
        
        # Opcional: Ver qué parámetros ganó
        # print(f"   Mejores Params: {model.named_steps['regressor'].get_params()}")
    else:
        print(f"Modelo optimizado para {target} no encontrado.")

Error: No se encuentra 'test.csv'.

Resultados Finales en TEST (Usando Mejores Hiperparámetros):
------------------------------------------------------------
Target: NextClose_BTC1  | RMSE Test: 1,924.81 | MAPE Test: 2.60%
Target: NextClose_BTC2  | RMSE Test: 2,282.84 | MAPE Test: 3.34%
Target: NextClose_BTC3  | RMSE Test: 2,365.55 | MAPE Test: 3.51%
Target: NextClose_BTC4  | RMSE Test: 2,508.76 | MAPE Test: 3.71%
Target: NextClose_BTC5  | RMSE Test: 2,844.11 | MAPE Test: 4.33%
Target: NextClose_BTC6  | RMSE Test: 2,921.53 | MAPE Test: 4.35%
Target: NextClose_BTC7  | RMSE Test: 3,078.34 | MAPE Test: 4.63%
